In [27]:
from sklearn.model_selection import train_test_split
import numpy as np
import os
import tensorflow as tf
print(os.getcwd())
data_path = os.path.join(os.getcwd(), 'data')
X = np.load(os.path.join(data_path, 'X_69_200.npy'), allow_pickle=True)
# reshape X to be [n][f][t] f is frequency, t is time of the stft
freq_bins = X[0].shape[0]
data_size = X.shape[0]
# X = np.reshape(X, newshape=(data_size, nfft_bins,None ))
print(data_size, freq_bins)
Y = np.load(os.path.join(data_path, 'Y_69_200.npy'), allow_pickle=True)
# create tensor shape (data_size, nfft_bins, None) for X

# X = tf.ragged.constant(X)
# Y = tf.ragged.constant(Y)
X = [tf.RaggedTensor.from_uniform_row_length(x, uniform_row_length=freq_bins).flat_values for x in X]
Y = [tf.RaggedTensor.from_uniform_row_length(y, uniform_row_length=freq_bins).flat_values for y in Y]
splits = train_test_split(X, Y, test_size=0.2, random_state=69)
# test = [tf.RaggedTensor.from_uniform_row_length(x, uniform_row_length=freq_bins).flat_values for x in splits[0]]

X_train, X_test, y_train, y_test = splits
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

c:\Users\Julio\Documents\Projects\audio-denoiser
200 513


In [28]:

import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, BatchNormalizationV2, GlobalMaxPooling2D
from keras.layers import Flatten, Dropout, AveragePooling2D
from keras.layers import Conv2D, MaxPooling2D, Reshape
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import adam_v2
from keras import layers
import keras

# import rms prop from keras
# A Keras tensor is a symbolic tensor-like object, which we augment with certain attributes that allow us to build a Keras model just by knowing the inputs and outputs of the model.
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=100)
opt = adam_v2.Adam(learning_rate=0.0007)
# model.compile(loss='categorical_crossentropy', optimizer=opt)

# inputs are 2d arrays of shape [nfft_bins, time], time is a varried
lookahead = 1
frequency_bins = X_train[0].shape[0]
data_shape = (frequency_bins, None)
model_input = layers.Input(shape=(frequency_bins, None), dtype=tf.complex64)
print(model_input.shape)
# add channel dimension
x_imag = tf.math.imag(model_input)
# print(x_imag.shape)
x = tf.abs(model_input)
x = tf.expand_dims(x, axis=-1)
x = layers.Conv2D(filters=16, kernel_size=(frequency_bins//4,1),  padding='same', activation='relu')(x)
print(x.shape)
x = layers.Conv2D(filters=32, kernel_size=(frequency_bins//2,1),  padding='same', activation='relu')(x)
print(x.shape)
x = layers.Conv2D(filters=64, kernel_size=(1,1), padding='same', activation='relu')(x)
print(x.shape)
x = layers.Conv2D(filters=32, kernel_size=(frequency_bins//2,1),  padding='same', activation='relu')(x)
print(x.shape)
x = layers.Conv2D(filters=16, kernel_size=(frequency_bins//4,1),  padding='same', activation='relu')(x)
x = layers.Conv2D(filters=1, kernel_size=(1,1),  padding='same', activation='relu')(x)
# x = layers.GlobalMaxPooling2D(keepdims=True)(x)
# out = Dense(frequency_bins, activation="relu")(x)
x = tf.squeeze(x, axis=-1)
out = tf.complex(x, x_imag)
model = keras.Model(inputs=model_input, outputs=out)
model.summary()
model.compile(loss='mse',
              optimizer=opt, metrics=['accuracy'])

# model.add(Dropout(0.2))


# model.add(Dropout(0.2))

# model.add(Dropout(0.2))


# model.add(BatchNormalizationV2())

# model.add(GlobalMaxPooling2D())
# model.add(Flatten())



(None, 513, None)
(None, 513, None, 16)
(None, 513, None, 32)
(None, 513, None, 64)
(None, 513, None, 32)
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 513, None)]  0           []                               
                                                                                                  
 tf.math.abs_4 (TFOpLambda)     (None, 513, None)    0           ['input_5[0][0]']                
                                                                                                  
 tf.expand_dims_4 (TFOpLambda)  (None, 513, None, 1  0           ['tf.math.abs_4[0][0]']          
                                )                                                                 
                                                                                     

In [167]:
x_list = [*X[:5]]
print(x_list)
# xrag = tf.ragged.constant([*X[:5]], ragged_rank=1)

[array([[-2.6253945e-01+0.0000000e+00j, -3.1784039e-02+0.0000000e+00j,
         2.9991561e-01+0.0000000e+00j, ...,
        -2.0312765e-01+0.0000000e+00j,  1.7549641e-01+0.0000000e+00j,
         2.5087645e+00+0.0000000e+00j],
       [ 2.1812922e-01-4.6569560e-02j,  5.7326477e-02+3.0661318e-01j,
        -1.4808489e-01+9.5876336e-02j, ...,
        -6.7454316e-02-4.5057178e-01j, -6.8061954e-01-6.9144595e-01j,
        -3.2846963e+00-2.5067531e-02j],
       [-1.4768565e-01-1.4486320e-01j,  3.2243554e-02-5.9185648e-01j,
        -1.7507219e-01-1.3448322e-02j, ...,
        -5.2864468e-01+3.1881273e+00j,  1.3874222e+00-1.3338257e+00j,
         4.2838950e+00+1.9844971e+00j],
       ...,
       [-5.6475759e-03+2.4403463e-04j,  8.1788318e-07+2.8091998e-07j,
        -1.6480251e-07-2.7836819e-07j, ...,
        -3.9473682e-07+3.3293509e-07j,  9.6426436e-07+1.3906350e-06j,
        -4.0004823e-02-7.9928357e-03j],
       [ 5.6475112e-03-1.2262944e-04j, -1.5429889e-06+5.3898117e-07j,
        -5.5109354e-0

In [29]:
cnnhistory=model.fit((X_train), y_train, batch_size=100, epochs=200, callbacks=[es], verbose=2)
plt.plot(cnnhistory.history['loss'])
plt.plot(cnnhistory.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

Epoch 1/200


ValueError: in user code:

    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Python310\lib\site-packages\keras\engine\input_spec.py", line 216, in assert_input_compatibility
        raise ValueError(

    ValueError: Layer "model_2" expects 1 input(s), but it received 160 input tensors. Inputs rec